## ML-第五版本-类别特征编码-时间序列特征-交叉累计特征
+ 类别特征+时间序列特征+累计交叉特征+lightgbm: 训练集AUC均值：0.9447834372991226, 训练集合AUC方差: 0.0017872366333431303
+ 类别特征+时间序列特征+累计交叉特征+null importance+lightgbm: 

## 导包

In [1]:
import os
import re
import numpy as np
import pandas as pd
import datetime
import pickle as pkl

import matplotlib.pyplot as plt
import seaborn as sns

import scorpyo as sp
from null_importance import get_null_importance
from time_sequence_feats import get_time_base, get_sequence_statis, get_sequence_groupby_statis
import category_encoders as ce

from gensim.models import word2vec

import time
from sklearn.metrics import roc_auc_score as auc
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, KFold

import warnings
warnings.filterwarnings("ignore")

pd.set_option('max_rows', 320, 'max_columns',100)

In [2]:
path_project = r'/Users/liliangshan/workspace/python/01_datasets/ccf_system_access_risk_identification'

# path dir
path_row_data = os.path.join(path_project, 'row_data')
path_new_data = os.path.join(path_project, 'new_data')
path_results  = os.path.join(path_project, 'results')
path_results_version  = os.path.join(path_results, 'version_2')

# path row_data
path_train = os.path.join(path_row_data, 'train.csv')
path_test  = os.path.join(path_row_data, 'evaluation_public.csv')
path_sample_submission = os.path.join(path_row_data, 'submit_example.csv')

## results
path_output_report = os.path.join(path_results, '01_原始数据探察_20221014.xlsx')

y_label = "is_risk"


# new_train_data
path_new_train_cate = os.path.join(path_new_data, 'train_cate.pkl')
path_new_test_cate  = os.path.join(path_new_data, 'test_cate.pkl')

path_new_train_time_sequence = os.path.join(path_new_data, 'train_time_sequence.pkl')
path_new_test_time_sequence  = os.path.join(path_new_data, 'test_time_sequence.pkl')

path_new_train_time_sequence_woe = os.path.join(path_new_data, 'train_time_sequence_woe.pkl')
path_new_test_time_sequence_woe  = os.path.join(path_new_data, 'test_time_sequence_woe.pkl')

path_new_train_cumsum = os.path.join(path_new_data, 'train_cumsum.pkl')
path_new_test_cumsum = os.path.join(path_new_data, 'test_cumsum.pkl')

path_new_train_cumsum_woe = os.path.join(path_new_data, 'train_cumsum_woe.pkl')
path_new_test_cumsum_woe = os.path.join(path_new_data, 'test_cumsum_woe.pkl')

path_new_train_embeding = os.path.join(path_new_data, 'train_embeding.pkl')
path_new_test_embeding = os.path.join(path_new_data, 'test_embeding.pkl')

In [3]:
df_row_train = pd.read_csv(path_train)
df_row_val  = pd.read_csv(path_test)

## 工具函数

In [4]:
def save_pkl(data, path_data):
    """将数据保存为pkl"""
    with open(path_data, 'wb') as f:
        pkl.dump(data, f)
        
def read_pkl(path_data):
    """读取pkl格式数据"""
    with open(path_data, 'rb') as f:
        data = pkl.load(f)
    return data

## 类别特征+时间序列特征+累计交叉特征+lightgbm

### 类别特征+时间序列特征+累计交叉特征

In [31]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_time_sequence = read_pkl(path_new_train_time_sequence)
df_val_time_sequence = read_pkl(path_new_test_time_sequence)

df_train_cumsum = read_pkl(path_new_train_cumsum)
df_val_cumsum = read_pkl(path_new_test_cumsum)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [32]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_time_sequence, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_time_sequence, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_train_cumsum, on='id')
df_row_val = pd.merge(left=df_row_val, right=df_val_cumsum, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [33]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,...,op_month_device_num_transform_op_city_苏州_error_code_cumsum,op_month_device_num_transform_log_system_transform_cumunique,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_cumsum,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_error_code_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_error_code_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_error_code_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_error_code_cumsum,op_month_device_num_transform_log_system_transform_nan_cumsum,op_month_device_num_transform_log_system_transform_nan_error_code_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_error_code_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_error_code_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_error_code_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_error_code_cumsum,op_month_device_num_transform_url_cumunique,op_month_device_num_transform_url_xxx.com/github_cumsum,op_month_device_num_transform_url_xxx.com/github_error_code_cumsum,op_month_device_num_transform_url_hr.xxx.com/_cumsum,op_month_device_num_transform_url_hr.xxx.com/_error_code_cumsum,op_month_device_num_transform_url_work.xxx.com/task_cumsum,op_month_device_num_transform_url_work.xxx.com/task_error_code_cumsum,op_month_device_num_transform_url_xxx.com/mail_cumsum,op_month_device_num_transform_url_xxx.com/mail_error_code_cumsum,op_month_device_num_transform_url_xxx.com/oa_cumsum,op_month_device_num_transform_url_xxx.com/oa_error_code_cumsum,op_month_device_num_transform_url_xxx.com/getVerifyCode_cumsum,op_month_device_num_transform_url_xxx.com/getVerifyCode_error_code_cumsum,op_month_device_num_transform_url_xxx.com/loginAuth_cumsum,op_month_device_num_transform_url_xxx.com/loginAuth_error_code_cumsum,op_month_device_num_transform_url_wpsdoc.xxx.com/kdocs_cumsum,op_month_device_num_transform_url_wpsdoc.xxx.com/kdocs_error_code_cumsum,op_month_device_num_transform_url_business.xxx.com/_cumsum,op_month_device_num_transform_url_business.xxx.com/_error_code_cumsum,op_month_device_num_transform_url_xxx.com/checkingin_cumsum,op_month_device_num_transform_url_xxx.com/checkingin_error_code_cumsum,op_month_device_num_transform_url_xxx.com/getLoginTyp

### 获取建模使用的特征

In [10]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [34]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

### lightgbm

In [15]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [16]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [17]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.939554	valid_1's auc: 0.934803
[20]	training's auc: 0.945894	valid_1's auc: 0.939021
[30]	training's auc: 0.953318	valid_1's auc: 0.94137
[40]	training's auc: 0.959774	valid_1's auc: 0.942402
[50]	training's auc: 0.965208	valid_1's auc: 0.942317
[60]	training's auc: 0.971102	valid_1's auc: 0.942433
[70]	training's auc: 0.97875	valid_1's auc: 0.941518
[80]	training's auc: 0.982856	valid_1's auc: 0.940801
[90]	training's auc: 0.987169	valid_1's auc: 0.939203
[100]	training's auc: 0.989307	valid_1's auc: 0.93976
[110]	training's auc: 0.99121	valid_1's auc: 0.940287
[120]	training's auc: 0.992714	valid_1's auc: 0.940222
[130]	training's auc: 0.993718	valid_1's auc: 0.939611
Early stopping, best iteration is:
[37]	training's auc: 0.956695	valid_1's auc: 0.943511
----------- 1
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.939642	valid_1's auc: 0.935284
[20

,name,importance
1086,system_system_op_times_groups_http_status_code...,106433.265059
84,max_ip_transform_system_op_diff_second1_6,32952.971625
86,std_ip_transform_system_op_diff_second1_6,13899.005156
1239,system_op_days_http_status_code_200_cumsum,10266.198241
117,avg_user_name_system_op_diff_second1_6,5701.874188
20,op_datetime_hour,4899.893189
33,op_datetime_hour_cos,4276.925064
82,avg_ip_transform_system_op_diff_second1_6,3296.073467
1627,op_times_groups_ip_transform_url_wpsdoc.xxx.co...,1982.791257
130,kurt_user_name_system_op_diff_second1_9,1440.426911


训练集AUC均值：0.9447834372991226, 训练集合AUC方差: 0.0017872366333431303


## 类别特征+时间序列特征+累计交叉特征+null importance+lightgbm 

### null importance

In [18]:
feats, categorical_feats = get_null_importance(df_train.drop(columns=[y_label,'id']).copy(),
                                               df_train[y_label].copy(), 
                                               thresholds=10)

In [19]:
categorical_feats

[]

### lightgbm

In [21]:
# feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [35]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [36]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

KeyError: "['oof'] not in index"

## 类别特征+时间序列特征WOE+累计交叉特征WOE+lightgbm

### 类别特征+时间序列特征WOE+累计交叉特征WOE

In [5]:
df_train_cate = read_pkl(path_new_train_cate)
df_val_cate = read_pkl(path_new_test_cate)

df_train_time_sequence_woe = read_pkl(path_new_train_time_sequence_woe)
df_val_time_sequence_woe = read_pkl(path_new_test_time_sequence_woe)

df_train_cumsum_woe = read_pkl(path_new_train_cumsum_woe)
df_val_cumsum_woe = read_pkl(path_new_test_cumsum_woe)

df_raw_train = pd.read_csv(path_train)
df_raw_val  = pd.read_csv(path_test)

In [6]:
df_row_train = pd.merge(left=df_train_cate, right=df_train_time_sequence_woe, on='id')
df_row_val = pd.merge(left=df_val_cate, right=df_val_time_sequence_woe, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_train_cumsum_woe, on='id')
df_row_val = pd.merge(left=df_row_val, right=df_val_cumsum_woe, on='id')

df_row_train = pd.merge(left=df_row_train, right=df_raw_train[['id', y_label]], on='id')
df_row_val = pd.merge(left=df_row_val, right=df_raw_val[['id']], on='id')

In [7]:
df_row_train.head()

,id,user_name,department,ip_transform,device_num_transform,browser_version,browser,os_type,os_version,ip_type,http_status_code,op_city,log_system_transform,url,op_diff_second1,op_diff_second2,system_op_diff_second1,system_op_diff_second2,op_datetime_year,op_datetime_month,op_datetime_day,op_datetime_hour,op_datetime_minute,op_datetime_second,op_datetime_quarter,op_datetime_dayofweek,op_datetime_is_year_start,op_datetime_is_month_start,op_datetime_is_month_end,op_datetime_second_sin,op_datetime_second_cos,op_datetime_minute_sin,op_datetime_minute_cos,op_datetime_hour_sin,op_datetime_hour_cos,op_datetime_day_sin,op_datetime_day_cos,op_datetime_dayofweek_sin,op_datetime_dayofweek_cos,op_datetime_month_sin,op_datetime_month_cos,avg_3_system_op_diff_second1,median_3_system_op_diff_second1,max_3_system_op_diff_second1,min_3_system_op_diff_second1,std_3_system_op_diff_second1,skew_3_system_op_diff_second1,kurt_3_system_op_diff_second1,avg_6_system_op_diff_second1,median_6_system_op_diff_second1,...,op_month_device_num_transform_op_city_苏州_error_code_cumsum,op_month_device_num_transform_log_system_transform_cumunique,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_cumsum,op_month_device_num_transform_log_system_transform_nHrKgKdJ1Mzt_error_code_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_cumsum,op_month_device_num_transform_log_system_transform_a5G25puBl9xj_error_code_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_cumsum,op_month_device_num_transform_log_system_transform_sW0whYIx8LFM_error_code_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_cumsum,op_month_device_num_transform_log_system_transform_2umVQwhiiwNJ_error_code_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_cumsum,op_month_device_num_transform_log_system_transform_fwM6KZKjrzjm_error_code_cumsum,op_month_device_num_transform_log_system_transform_nan_cumsum,op_month_device_num_transform_log_system_transform_nan_error_code_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_cumsum,op_month_device_num_transform_log_system_transform_dwS3cdn15GK4_error_code_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_cumsum,op_month_device_num_transform_log_system_transform_RwHe8Q1R7AlB_error_code_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_cumsum,op_month_device_num_transform_log_system_transform_9RAS6RNfETj5_error_code_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_cumsum,op_month_device_num_transform_log_system_transform_2UNHLdxlhIzv_error_code_cumsum,op_month_device_num_transform_url_cumunique,op_month_device_num_transform_url_xxx.com/github_cumsum,op_month_device_num_transform_url_xxx.com/github_error_code_cumsum,op_month_device_num_transform_url_hr.xxx.com/_cumsum,op_month_device_num_transform_url_hr.xxx.com/_error_code_cumsum,op_month_device_num_transform_url_work.xxx.com/task_cumsum,op_month_device_num_transform_url_work.xxx.com/task_error_code_cumsum,op_month_device_num_transform_url_xxx.com/mail_cumsum,op_month_device_num_transform_url_xxx.com/mail_error_code_cumsum,op_month_device_num_transform_url_xxx.com/oa_cumsum,op_month_device_num_transform_url_xxx.com/oa_error_code_cumsum,op_month_device_num_transform_url_xxx.com/getVerifyCode_cumsum,op_month_device_num_transform_url_xxx.com/getVerifyCode_error_code_cumsum,op_month_device_num_transform_url_xxx.com/loginAuth_cumsum,op_month_device_num_transform_url_xxx.com/loginAuth_error_code_cumsum,op_month_device_num_transform_url_wpsdoc.xxx.com/kdocs_cumsum,op_month_device_num_transform_url_wpsdoc.xxx.com/kdocs_error_code_cumsum,op_month_device_num_transform_url_business.xxx.com/_cumsum,op_month_device_num_transform_url_business.xxx.com/_error_code_cumsum,op_month_device_num_transform_url_xxx.com/checkingin_cumsum,op_month_device_num_transform_url_xxx.com/checkingin_error_code_cumsum,op_month_device_num_transform_url_xxx.com/getLoginTyp

### 获取建模使用的特征

In [8]:
remove_cols = ['id']
feats = df_row_train.columns
feats = feats.drop(remove_cols)
feats = feats.drop(y_label)

In [9]:
df_train = df_row_train.reset_index(drop=True)
df_test  = df_row_val.reset_index(drop=True)

### lightgbm

In [13]:
feats = feats_importance.sort_values('importance', ascending=False)[:50].name.values

In [14]:
params = {
    'learning_rate': 0.05,
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'num_leaves': 31,
    'verbose': -1,
    'seed': 2222,
    'n_jobs': -1,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.9,
    'bagging_freq': 4,
    # 'min_child_weight': 10,
}

res_sort_col = 'id'

In [15]:
fold_num = 5
seeds = [2022]
oof = np.zeros(len(df_train))
importance = 0
pred_y = pd.DataFrame()
score = []


for seed in seeds:
    kf = StratifiedKFold(n_splits=fold_num, shuffle=True, random_state=seed)
    for fold, (train_idx, val_idx) in enumerate(kf.split(df_train[feats], df_train[y_label])):
        print('-----------', fold)
        # 数据集构造
        train = lgb.Dataset(df_train.loc[train_idx, feats],
                            df_train.loc[train_idx, y_label]
                           )
        val = lgb.Dataset(df_train.loc[val_idx, feats],
                          df_train.loc[val_idx, y_label]
                         )
        # 模型训练
        model = lgb.train(params, train, valid_sets=[train, val], 
                          num_boost_round=20000, early_stopping_rounds=100, verbose_eval=10)
        
        # 结果及得分获取
        oof[val_idx] += model.predict(df_train.loc[val_idx, feats]) / len(seeds)
        pred_y['fold_%d_seed_%d' % (fold, seed)] = model.predict(df_test[feats])
        importance += model.feature_importance(importance_type='gain') / fold_num
        score.append(auc(df_train.loc[val_idx, y_label], model.predict(df_train.loc[val_idx, feats])))

# 特征重要性获取
feats_importance = pd.DataFrame()
feats_importance['name'] = feats
feats_importance['importance'] = importance
display(feats_importance.sort_values('importance', ascending=False)[:30])

# 训练集得分效果
df_train['oof'] = oof
print('训练集AUC均值：{}, 训练集合AUC方差: {}'.format(np.mean(score), np.std(score)))

# 测试集合结果输出
score = np.mean(score)
df_test[y_label] = pred_y.mean(axis=1).values
df_test = df_test.sort_values(res_sort_col).reset_index(drop=True)

if not os.path.exists(path_results_version):
    os.makedirs(path_results_version)

# 结果保存
sub = pd.read_csv(path_sample_submission)
sub[y_label] = df_test[y_label].values
sub.to_csv(os.path.join(path_results_version,time.strftime('lgb_%Y%m%d%H%M_')+'%.5f.csv'%score), index=False)

----------- 0
Training until validation scores don't improve for 100 rounds
[10]	training's auc: 0.998133	valid_1's auc: 0.99486
[20]	training's auc: 0.99907	valid_1's auc: 0.996907
[30]	training's auc: 0.999483	valid_1's auc: 0.998205
[40]	training's auc: 0.999656	valid_1's auc: 0.999118
[50]	training's auc: 0.999783	valid_1's auc: 0.999253
[60]	training's auc: 0.999846	valid_1's auc: 0.99944
[70]	training's auc: 0.999905	valid_1's auc: 0.999579
[80]	training's auc: 0.999938	valid_1's auc: 0.999654
[90]	training's auc: 0.999965	valid_1's auc: 0.999694
[100]	training's auc: 0.999981	valid_1's auc: 0.999707
[110]	training's auc: 0.999992	valid_1's auc: 0.999747
[120]	training's auc: 0.999997	valid_1's auc: 0.999771
[130]	training's auc: 0.999999	valid_1's auc: 0.999784
[140]	training's auc: 1	valid_1's auc: 0.999799
[150]	training's auc: 1	valid_1's auc: 0.99981
[160]	training's auc: 1	valid_1's auc: 0.99982
[170]	training's auc: 1	valid_1's auc: 0.999839
[180]	training's auc: 1	valid_1

[270]	training's auc: 1	valid_1's auc: 0.999909
[280]	training's auc: 1	valid_1's auc: 0.999913
[290]	training's auc: 1	valid_1's auc: 0.999913
[300]	training's auc: 1	valid_1's auc: 0.999915
[310]	training's auc: 1	valid_1's auc: 0.999915
[320]	training's auc: 1	valid_1's auc: 0.999915
[330]	training's auc: 1	valid_1's auc: 0.999919
[340]	training's auc: 1	valid_1's auc: 0.99992
[350]	training's auc: 1	valid_1's auc: 0.999921
[360]	training's auc: 1	valid_1's auc: 0.999924
[370]	training's auc: 1	valid_1's auc: 0.999924
[380]	training's auc: 1	valid_1's auc: 0.999924
[390]	training's auc: 1	valid_1's auc: 0.999926
[400]	training's auc: 1	valid_1's auc: 0.999926
[410]	training's auc: 1	valid_1's auc: 0.999927
[420]	training's auc: 1	valid_1's auc: 0.999929
[430]	training's auc: 1	valid_1's auc: 0.99993
[440]	training's auc: 1	valid_1's auc: 0.999929
[450]	training's auc: 1	valid_1's auc: 0.999932
[460]	training's auc: 1	valid_1's auc: 0.999932
[470]	training's auc: 1	valid_1's auc: 0.9

[150]	training's auc: 1	valid_1's auc: 0.999726
[160]	training's auc: 1	valid_1's auc: 0.99974
[170]	training's auc: 1	valid_1's auc: 0.999756
[180]	training's auc: 1	valid_1's auc: 0.99976
[190]	training's auc: 1	valid_1's auc: 0.999765
[200]	training's auc: 1	valid_1's auc: 0.999778
[210]	training's auc: 1	valid_1's auc: 0.999782
[220]	training's auc: 1	valid_1's auc: 0.99979
[230]	training's auc: 1	valid_1's auc: 0.999803
[240]	training's auc: 1	valid_1's auc: 0.999806
[250]	training's auc: 1	valid_1's auc: 0.999813
[260]	training's auc: 1	valid_1's auc: 0.99982
[270]	training's auc: 1	valid_1's auc: 0.999827
[280]	training's auc: 1	valid_1's auc: 0.999832
[290]	training's auc: 1	valid_1's auc: 0.999833
[300]	training's auc: 1	valid_1's auc: 0.999837
[310]	training's auc: 1	valid_1's auc: 0.999837
[320]	training's auc: 1	valid_1's auc: 0.999838
[330]	training's auc: 1	valid_1's auc: 0.999841
[340]	training's auc: 1	valid_1's auc: 0.99984
[350]	training's auc: 1	valid_1's auc: 0.9998

,name,importance
0,system_op_month_http_status_code_200_cumsum,41923.961660
1,max_6_system_op_diff_second1,39191.832131
2,system_op_month_cumsum,31059.713555
5,max_ip_transform_system_op_diff_second2_3,30730.258425
3,system_system_op_times_groups_http_status_code...,28586.410807
4,avg_ip_transform_system_op_diff_second2_3,26863.675603
6,system_op_month_ip_type_内网_cumsum,16971.077772
15,max_ip_type_system_op_diff_second1_6,12361.311380
7,system_op_month_department_rd_cumsum,11629.602391
8,skew_ip_transform_system_op_diff_second1_6,9650.348692


训练集AUC均值：0.9999347648674389, 训练集合AUC方差: 3.0709910084691385e-05
